## Integrantes

1. 
2. 
3. 
4. 

# Instrucciones

El laboratorio tiene 6ptos, donde obtener 6 ptos equivale a un 7.0 y 0 ptos a un 1.0. El formato de entrega será subir a u-cursos un **único** jupyter notebook por grupo.
El notebook debe ejecutar desde la primera celda a la última **sin errores**. Todo el código debe estar en el mismo notebook. El código debe estar en un formato **presentable** y **ejecutado**.


**Nota:** Se descontará puntaje si adjunta gráficos sin descripción. Para esto puede añadir una descripción en los atributos xlabel, ylabel y título; o añadir una celda markdown debajo del gráfico de la forma "**Figura x:** descripción detallada...".

# Problema

Neural Information Processing Systems (NIPS) es una de las conferencias más importantes sobre machine learning en el mundo. El objetivo de este laboratorio es hallar los temás presentes en las publicaciones científicas (papers) presentados en esta conferencia, de esta manera una persona no muy entendida en el tema de machine learning puede tener una idea de las distintas líneas de investigación que existen. La base de datos con la que cuenta para repsonder esta pregunta incluye títulos, autores, abstract, y el texto presente de todos los papers del NIPS desde 1987 hasta el 2016. Por simplicidad, se trabaja solo con los abstract de los papers que son guardados en la variable **corpus**.

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("data/papers.csv")
df.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [18]:
corpus = df[df["abstract"]!='Abstract Missing']["abstract"].tolist()
corpus[0]

'Non-negative matrix factorization (NMF) has previously been shown to \r\nbe a useful decomposition for multivariate data. Two different multi- \r\nplicative algorithms for NMF are analyzed. They differ only slightly in \r\nthe multiplicative factor used in the update rules. One algorithm can be \r\nshown to minimize the conventional least squares error while the other \r\nminimizes the generalized Kullback-Leibler divergence. The monotonic \r\nconvergence of both algorithms can be proven using an auxiliary func- \r\ntion analogous to that used for proving convergence of the Expectation- \r\nMaximization algorithm. The algorithms can also be interpreted as diag- \r\nonally rescaled gradient descent, where the rescaling factor is optimally \r\nchosen to ensure convergence. '

# A. Investigación (1.5 ptos)

## A.1 ¿Que es el modelado de tópicos? (0.75 ptos)

Explique en que consiste el modelado de tópicos, para qué sirve y de dos ejemplos de uso distintos al problema del laboratorio.

## A.2 ¿Qué es LDA? (0.75 pto)

Explique en palabras qué es LDA, qué es $K$, $\phi_{k}$ y $\pi_{d}$.

# B. Preparación (2.5 ptos)

Esta sección tiene por objetivo crear una base de datos consolidada para el uso de modelos de tópicos.

## B.1 Análisis exploratorio inicial (0.25 ptos)

Compute la cantidad de tokens y el tamaño del vocabulario del corpus sin realizar ningún tipo de procesamiento.

## B.2 Limpieza de los datos (1.0 pto)

Cree una función de tokenización que procese los papers. Escoja un ejemplo del conjunto de datos y muestra como se ve antes y despues de aplicar el procesamiento. Esta función debe eliminar patrónes de caracteres característicos del corpus que no aportan información (por ejemplo: "\r") y eliminar stopwords que se encuentran en el archivo adjunto (**stopwords.txt**). Compute la cantidad de tokens y el tamaño del vocabulario resultante de aplicar este nivel de procesamiento al corpus. ¿Por qué se deben eliminar las stopwords?

**Hint**: para eliminar patrónes de caracteres puede usar la función **re.sub()**.

## B.3 Post procesamiento (1.0)

Realice un postprocesamietno del corpus que sirva para reducir aún más el vocabulario: análisis de palabras muy frecuentes, poco frecuentes, errores ortográficos, etc. Compute la cantidad de tokens y el tamaño del vocabulario resultante de aplicar este nivel de procesamiento al corpus. Complete el cuadro resumen de estadísticas.

|Etapa|tokens|vocabulario|
|---|---|---|
|A.1|||
|A.2|||
|A.3|||

# C. LDA (2ptos)

En este sección tiene por objetivo descubrir los tópicos presentes en el corpus. Documentación de LDA:

https://radimrehurek.com/gensim/models/ldamodel.html

## C.1 Corpus (0.5 ptos)

Genere el corpus el formato que lda requiere e imprima un ejemplo de un documento en dicho formato y en el formato original.

In [ ]:
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

## C.2  Interpretación de los tópicos (1.5 ptos)
Entrene LDA para que encuentre 10 tópicos ($K=10$). Luego, interprete los tópicos encontrados. Para esto se puede apollar del método **.show_topics**  o de la visualización de **pyLDAvis**. Responde a la siguiente pregunta: ¿Cuáles son los tópicos más relevantes?

In [ ]:
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim